x日だけをREG率順に表示

In [12]:
import pandas as pd
import os
from datetime import datetime

# --- 1) 日付とディレクトリ設定 ---
date_str = "2025-11-08"  # YYYY-MM-DD 形式で変更
output_dir = r"C:\Users\akafu\analytics\aim_machi\data"  # CSVのディレクトリ

# --- 2) 曜日を自動で取得してファイル名を組み立て ---
weekday = datetime.strptime(date_str, "%Y-%m-%d").strftime("%a")  # Mon, Tue...
filename = f"aim-{date_str}-{weekday}.csv"
file_path = os.path.join(output_dir, filename)

# --- 3) CSV 読み込み ---
df = pd.read_csv(file_path)

# --- 4) 「1/xxx」表記を数値化 ---
df['RB_denominator'] = df['RB率'].str.extract(r'1/(\d+)').astype(float)
df['RB_frequency'] = 1 / df['RB_denominator']

# --- 5) RB_frequency で降順ソート ---
df_sorted = df.sort_values('RB_frequency', ascending=False)

# --- 6) 上位20件を表示 ---
df_sorted.head(20)


,台番,差枚,G数,出率,BB,RB,合成,BB率,RB率,RB_denominator,RB_frequency
66,699,923,8004,103.8,26,42,1/118,1/308,1/191,191.0,0.005236
39,603,1774,7510,107.9,30,39,1/109,1/250,1/193,193.0,0.005181
1,545,7580,9858,125.6,58,51,1/90,1/170,1/193,193.0,0.005181
35,599,747,7427,103.4,24,37,1/122,1/309,1/201,201.0,0.004975
37,601,3713,9347,113.2,38,46,1/111,1/246,1/203,203.0,0.004926
29,593,1450,6109,107.9,25,29,1/113,1/244,1/211,211.0,0.004739
16,560,874,8982,103.2,30,39,1/130,1/299,1/230,230.0,0.004348
0,544,3181,9458,111.2,41,41,1/115,1/231,1/231,231.0,0.004329
28,592,361,6836,101.8,23,29,1/131,1/297,1/236,236.0,0.004237
59,692,-380,6403,98.0,19,27,1/139,1/337,1/237,237.0,0.004219


条件つき(一日)

In [6]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data2"            # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"  # 出力先（まとめCSV）
START_DATE = "2025-7-1"    # ←開始日（"2025-07-01" でもOK）
END_DATE   = "2025-11-19"  # ←終了日

# 抽出条件
G_MIN = 2000
RATE_MAX_DEN = 10000   # REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 4000     # 差枚 > 4000

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== アイムジャグラーの定数（ブドウ逆算用） =====
COIN_IN_PER_GAME = 3
BIG_PAYOUT       = 252
REG_PAYOUT       = 96

REPLAY_PROB    = 1 / 7.298
REPLAY_PAYOUT  = 3

CHERRY_PROB    = 1 / 53.43
CHERRY_PAYOUT  = 2

BELL_PROB      = 1 / 4915.20
BELL_PAYOUT    = 14

PIERROT_PROB   = 1 / 10865.18
PIERROT_PAYOUT = 10

GRAPE_PAYOUT   = 8  # ぶどう 8枚役


def calc_grape_prob_denom(row: pd.Series):
    """
    1行ぶんから「ぶどう確率の分母」（例: 6.15 → 1/6.15）を計算して返す。
    計算不可能な場合は np.nan を返す。
    """
    spins = pd.to_numeric(row.get("G数"),  errors="coerce")
    bb    = pd.to_numeric(row.get("BB"),   errors="coerce")
    rb    = pd.to_numeric(row.get("RB"),   errors="coerce")
    diff  = pd.to_numeric(row.get("差枚"), errors="coerce")

    if (
        pd.isna(spins) or pd.isna(bb) or pd.isna(rb) or pd.isna(diff) or
        spins <= 0
    ):
        return np.nan

    total_in  = spins * COIN_IN_PER_GAME
    total_out = total_in + diff

    # ボーナス払い出し
    bonus_out       = bb * BIG_PAYOUT + rb * REG_PAYOUT
    # 小役トータル払い出し
    small_out_total = total_out - bonus_out

    # ぶどう以外の小役払い出し（理論値）
    other_small_out = spins * (
        REPLAY_PROB   * REPLAY_PAYOUT +
        CHERRY_PROB   * CHERRY_PAYOUT +
        BELL_PROB     * BELL_PAYOUT   +
        PIERROT_PROB  * PIERROT_PAYOUT
    )

    grape_out = small_out_total - other_small_out
    if grape_out <= 0:
        return np.nan

    grape_hits = grape_out / GRAPE_PAYOUT
    if grape_hits <= 0:
        return np.nan

    grape_prob = spins / grape_hits   # 1/◯ の ◯
    return grape_prob


# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    """
    data2 内のファイル名を柔軟に探す:
      - aim-YYYY-MM-DD-曜_with_grape.csv
      - aim-YYYY-MM-DD-曜.csv
      - aim-YYYY-M-D-曜_with_grape.csv
      - aim-YYYY-M-D-曜.csv
    の順で存在チェックして、最初に見つかったものを返す。
    どれもなければ最初の候補パスを返す（read_day 側で FileNotFoundError）。
    """
    wd = weekday_str(d)
    date_pad    = d.strftime("%Y-%m-%d")               # 例: 2025-09-26
    date_nopad  = f"{d.year}-{d.month}-{d.day}"        # 例: 2025-9-26

    candidates = [
        f"aim-{date_pad}-{wd}_with_grape.csv",
        f"aim-{date_pad}-{wd}.csv",
        f"aim-{date_nopad}-{wd}_with_grape.csv",
        f"aim-{date_nopad}-{wd}.csv",
    ]

    for fname in candidates:
        path = os.path.join(DATA_DIR, fname)
        if os.path.exists(path):
            return path

    # どれも無ければ一番最初の候補を返しておく（read_day 側でエラーにするため）
    return os.path.join(DATA_DIR, candidates[0])

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)         # カンマ除去
                 .str.replace(r"\s+", "", regex=True))      # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)          # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、rate_denominator と ブドウ率 を付与して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    # ブドウ率を計算して文字列列として追加（既存があっても上書きでOK）
    budo_denom = df.apply(calc_grape_prob_denom, axis=1)
    df["ブドウ率"] = budo_denom.map(
        lambda x: f"1/{x:.2f}" if pd.notna(x) else ""
    )

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] > SAMAI_UPPER).fillna(False)

    out = df.loc[cond_g & cond_rate & cond_s].copy()
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = [
        "台番", "差枚", "G数", "BB", "RB",
        "出率", "BB率", "RB率", "合成",
        "ブドウ率",            # ★ここが翌日のブドウ率 → ブドウ率_next になる
    ]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(
        columns={c: f"{c}_next" for c in exist_cols if c != "台番"}
    )

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)


[SKIP] aim-2025-09-16-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-17-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-18-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-19-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-21-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-22-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-23-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-24-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-25-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-28-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-29-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-30-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-01-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-02-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-03-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-05-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-06-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-07-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-08-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-09-Thu_with_

条件（期間）

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data2"                  # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"       # 出力先（まとめCSV）
START_DATE = "2025-6-13"   # ←開始日
END_DATE   = "2025-9-15"   # ←終了日

# 抽出条件
G_MIN = 4000
RATE_MAX_DEN = 260# REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 1000

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    # aim-YYYY-MM-DD-曜_with_grape.csv を読む
    return os.path.join(
        DATA_DIR,
        f"aim-{d.strftime('%Y-%m-%d')}-{weekday_str(d)}_with_grape.csv"
    )

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)          # カンマ除去
                 .str.replace(r"\s+", "", regex=True))       # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)           # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、率の分母抽出＋主要数値列を Int64 正規化して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] < SAMAI_UPPER).fillna(False)

    out = df.loc[cond_g & cond_rate & cond_s].copy()
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = ["台番", "差枚", "G数", "BB", "RB", "出率", "BB率", "RB率", "合成"]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(columns={c: f"{c}_next" for c in exist_cols if c != "台番"})

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)


[SKIP] aim-2025-09-16-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-17-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-18-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-19-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-21-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-22-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-23-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-24-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-25-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-26-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-28-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-29-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-30-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-01-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-02-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-03-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-04-Sat_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-05-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-06-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-07-Tue_with_

In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data2"                  # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"       # 出力先（まとめCSV）
START_DATE = "2025-7-1"   # ←開始日
END_DATE   = "2025-11-19"  # ←終了日

# 抽出条件
G_MIN = 2000
RATE_MAX_DEN = 10000   # REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 1000     # 差枚 > 4000

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    return os.path.join(DATA_DIR, f"aim-{d.strftime('%Y-%m-%d')}-{weekday_str(d)}.csv")

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)         # カンマ除去
                 .str.replace(r"\s+", "", regex=True))      # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)          # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、率の分母抽出＋主要数値列を Int64 正規化して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] > SAMAI_UPPER).fillna(False)

    out = df.loc[cond_g & cond_rate & cond_s].copy()
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = [
        "台番", "差枚", "G数", "BB", "RB",
        "出率", "BB率", "RB率", "合成",
        "ブドウ率",            # ★ここを追加
    ]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(
        columns={c: f"{c}_next" for c in exist_cols if c != "台番"}
    )

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)


[SKIP] aim-2025-07-01-Tue.csv が見つかりません。
[SKIP] aim-2025-07-02-Wed.csv が見つかりません。
[SKIP] aim-2025-07-03-Thu.csv が見つかりません。
[SKIP] aim-2025-07-04-Fri.csv が見つかりません。
[SKIP] aim-2025-07-05-Sat.csv が見つかりません。
[SKIP] aim-2025-07-06-Sun.csv が見つかりません。
[SKIP] aim-2025-07-07-Mon.csv が見つかりません。
[SKIP] aim-2025-07-08-Tue.csv が見つかりません。
[SKIP] aim-2025-07-09-Wed.csv が見つかりません。
[SKIP] aim-2025-07-10-Thu.csv が見つかりません。
[SKIP] aim-2025-07-11-Fri.csv が見つかりません。
[SKIP] aim-2025-07-12-Sat.csv が見つかりません。
[SKIP] aim-2025-07-13-Sun.csv が見つかりません。
[SKIP] aim-2025-07-14-Mon.csv が見つかりません。
[SKIP] aim-2025-07-15-Tue.csv が見つかりません。
[SKIP] aim-2025-07-16-Wed.csv が見つかりません。
[SKIP] aim-2025-07-17-Thu.csv が見つかりません。
[SKIP] aim-2025-07-18-Fri.csv が見つかりません。
[SKIP] aim-2025-07-19-Sat.csv が見つかりません。
[SKIP] aim-2025-07-20-Sun.csv が見つかりません。
[SKIP] aim-2025-07-21-Mon.csv が見つかりません。
[SKIP] aim-2025-07-22-Tue.csv が見つかりません。
[SKIP] aim-2025-07-23-Wed.csv が見つかりません。
[SKIP] aim-2025-07-24-Thu.csv が見つかりません。
[SKIP] aim-2025-07-25-Fri.csv が見つかりません。


In [5]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data2"            # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"  # 出力先（まとめCSV）
START_DATE = "2025-7-2"    # ←開始日（"2025-07-01" でもOK）
END_DATE   = "2025-11-19"  # ←終了日

# 抽出条件
G_MIN = 4000
RATE_MAX_DEN = 260   # REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 1000     # 差枚 > 4000

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== アイムジャグラーの定数（ブドウ逆算用） =====
COIN_IN_PER_GAME = 3
BIG_PAYOUT       = 252
REG_PAYOUT       = 96

REPLAY_PROB    = 1 / 7.298
REPLAY_PAYOUT  = 3

CHERRY_PROB    = 1 / 53.43
CHERRY_PAYOUT  = 2

BELL_PROB      = 1 / 4915.20
BELL_PAYOUT    = 14

PIERROT_PROB   = 1 / 10865.18
PIERROT_PAYOUT = 10

GRAPE_PAYOUT   = 8  # ぶどう 8枚役


def calc_grape_prob_denom(row: pd.Series):
    """
    1行ぶんから「ぶどう確率の分母」（例: 6.15 → 1/6.15）を計算して返す。
    計算不可能な場合は np.nan を返す。
    """
    spins = pd.to_numeric(row.get("G数"),  errors="coerce")
    bb    = pd.to_numeric(row.get("BB"),   errors="coerce")
    rb    = pd.to_numeric(row.get("RB"),   errors="coerce")
    diff  = pd.to_numeric(row.get("差枚"), errors="coerce")

    if (
        pd.isna(spins) or pd.isna(bb) or pd.isna(rb) or pd.isna(diff) or
        spins <= 0
    ):
        return np.nan

    total_in  = spins * COIN_IN_PER_GAME
    total_out = total_in + diff

    # ボーナス払い出し
    bonus_out       = bb * BIG_PAYOUT + rb * REG_PAYOUT
    # 小役トータル払い出し
    small_out_total = total_out - bonus_out

    # ぶどう以外の小役払い出し（理論値）
    other_small_out = spins * (
        REPLAY_PROB   * REPLAY_PAYOUT +
        CHERRY_PROB   * CHERRY_PAYOUT +
        BELL_PROB     * BELL_PAYOUT   +
        PIERROT_PROB  * PIERROT_PAYOUT
    )

    grape_out = small_out_total - other_small_out
    if grape_out <= 0:
        return np.nan

    grape_hits = grape_out / GRAPE_PAYOUT
    if grape_hits <= 0:
        return np.nan

    grape_prob = spins / grape_hits   # 1/◯ の ◯
    return grape_prob


# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    """
    data2 内のファイル名を柔軟に探す:
      - aim-YYYY-MM-DD-曜_with_grape.csv
      - aim-YYYY-MM-DD-曜.csv
      - aim-YYYY-M-D-曜_with_grape.csv
      - aim-YYYY-M-D-曜.csv
    の順で存在チェックして、最初に見つかったものを返す。
    どれもなければ最初の候補パスを返す（read_day 側で FileNotFoundError）。
    """
    wd = weekday_str(d)
    date_pad    = d.strftime("%Y-%m-%d")               # 例: 2025-09-26
    date_nopad  = f"{d.year}-{d.month}-{d.day}"        # 例: 2025-9-26

    candidates = [
        f"aim-{date_pad}-{wd}_with_grape.csv",
        f"aim-{date_pad}-{wd}.csv",
        f"aim-{date_nopad}-{wd}_with_grape.csv",
        f"aim-{date_nopad}-{wd}.csv",
    ]

    for fname in candidates:
        path = os.path.join(DATA_DIR, fname)
        if os.path.exists(path):
            return path

    # どれも無ければ一番最初の候補を返しておく（read_day 側でエラーにするため）
    return os.path.join(DATA_DIR, candidates[0])

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)         # カンマ除去
                 .str.replace(r"\s+", "", regex=True))      # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)          # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、rate_denominator と ブドウ率 を付与して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    # ブドウ率を計算して文字列列として追加（既存があっても上書きでOK）
    budo_denom = df.apply(calc_grape_prob_denom, axis=1)
    df["ブドウ率"] = budo_denom.map(
        lambda x: f"1/{x:.2f}" if pd.notna(x) else ""
    )

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] > SAMAI_UPPER).fillna(False)

    out = df.loc[cond_g & cond_rate & cond_s].copy()
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = [
        "台番", "差枚", "G数", "BB", "RB",
        "出率", "BB率", "RB率", "合成",
        "ブドウ率",            # ★ここが翌日のブドウ率 → ブドウ率_next になる
    ]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(
        columns={c: f"{c}_next" for c in exist_cols if c != "台番"}
    )

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)


[SKIP] aim-2025-09-16-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-17-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-18-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-19-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-21-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-22-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-23-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-24-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-25-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-28-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-29-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-30-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-01-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-02-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-03-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-05-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-06-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-07-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-08-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-09-Thu_with_

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data2"            # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"  # 出力先（まとめCSV）
START_DATE = "2025-6-27"    # ←開始日（"2025-07-02" でもOK）
END_DATE   = "2025-11-19"  # ←終了日

# 抽出条件
G_MIN = 4000
RATE_MAX_DEN = 2000      # REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = -2000      # 差枚 > 1000
BUDO_MAX_DEN = 6.6 # ★ ブドウ分母が 6.0 以下（= 1/6.0 以内）

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== アイムジャグラーの定数（ブドウ逆算用） =====
COIN_IN_PER_GAME = 3
BIG_PAYOUT       = 252
REG_PAYOUT       = 96

REPLAY_PROB    = 1 / 7.298
REPLAY_PAYOUT  = 3

CHERRY_PROB    = 1 / 53.43
CHERRY_PAYOUT  = 2

BELL_PROB      = 1 / 4915.20
BELL_PAYOUT    = 14

PIERROT_PROB   = 1 / 10865.18
PIERROT_PAYOUT = 10

GRAPE_PAYOUT   = 8  # ぶどう 8枚役


def calc_grape_prob_denom(row: pd.Series):
    """
    1行ぶんから「ぶどう確率の分母」（例: 6.15 → 6.15）を計算して返す。
    計算不可能な場合は np.nan を返す。
    """
    spins = pd.to_numeric(row.get("G数"),  errors="coerce")
    bb    = pd.to_numeric(row.get("BB"),   errors="coerce")
    rb    = pd.to_numeric(row.get("RB"),   errors="coerce")
    diff  = pd.to_numeric(row.get("差枚"), errors="coerce")

    if (
        pd.isna(spins) or pd.isna(bb) or pd.isna(rb) or pd.isna(diff) or
        spins <= 0
    ):
        return np.nan

    total_in  = spins * COIN_IN_PER_GAME
    total_out = total_in + diff

    # ボーナス払い出し
    bonus_out       = bb * BIG_PAYOUT + rb * REG_PAYOUT
    # 小役トータル払い出し
    small_out_total = total_out - bonus_out

    # ぶどう以外の小役払い出し（理論値）
    other_small_out = spins * (
        REPLAY_PROB   * REPLAY_PAYOUT +
        CHERRY_PROB   * CHERRY_PAYOUT +
        BELL_PROB     * BELL_PAYOUT   +
        PIERROT_PROB  * PIERROT_PAYOUT
    )

    grape_out = small_out_total - other_small_out
    if grape_out <= 0:
        return np.nan

    grape_hits = grape_out / GRAPE_PAYOUT
    if grape_hits <= 0:
        return np.nan

    grape_prob = spins / grape_hits   # 1/◯ の ◯（分母）
    return grape_prob


# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    """
    data2 内のファイル名を柔軟に探す:
      - aim-YYYY-MM-DD-曜_with_grape.csv
      - aim-YYYY-MM-DD-曜.csv
      - aim-YYYY-M-D-曜_with_grape.csv
      - aim-YYYY-M-D-曜.csv
    の順で存在チェックして、最初に見つかったものを返す。
    どれもなければ最初の候補パスを返す（read_day 側で FileNotFoundError）。
    """
    wd = weekday_str(d)
    date_pad    = d.strftime("%Y-%m-%d")               # 例: 2025-09-26
    date_nopad  = f"{d.year}-{d.month}-{d.day}"        # 例: 2025-9-26

    candidates = [
        f"aim-{date_pad}-{wd}_with_grape.csv",
        f"aim-{date_pad}-{wd}.csv",
        f"aim-{date_nopad}-{wd}_with_grape.csv",
        f"aim-{date_nopad}-{wd}.csv",
    ]

    for fname in candidates:
        path = os.path.join(DATA_DIR, fname)
        if os.path.exists(path):
            return path

    # どれも無ければ一番最初の候補を返しておく（read_day 側でエラーにするため）
    return os.path.join(DATA_DIR, candidates[0])

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)         # カンマ除去
                 .str.replace(r"\s+", "", regex=True))      # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)          # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、rate_denominator と ブドウ率 を付与して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    # ブドウ率を計算（数値分母＋表示用文字列）
    budo_denom = df.apply(calc_grape_prob_denom, axis=1)
    df["budo_denominator"] = budo_denom
    df["ブドウ率"] = budo_denom.map(
        lambda x: f"1/{x:.2f}" if pd.notna(x) else ""
    )

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚、ブドウ分母）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator", "budo_denominator"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g     = (df["G数"] >= G_MIN).fillna(False)
    cond_rate  = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s     = (df["差枚"] < SAMAI_UPPER).fillna(False)
    cond_budo  = (df["budo_denominator"] <= BUDO_MAX_DEN).fillna(False)  # ★ ぶどう分母 ≤ 6.0

    out = df.loc[cond_g & cond_rate & cond_s & cond_budo].copy()
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["budo_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = [
        "台番", "差枚", "G数", "BB", "RB",
        "出率", "BB率", "RB率", "合成",
        "ブドウ率",            # 翌日のブドウ率 → ブドウ率_next になる
    ]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(
        columns={c: f"{c}_next" for c in exist_cols if c != "台番"}
    )

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)


[SKIP] aim-2025-09-16-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-17-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-18-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-19-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-21-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-22-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-23-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-24-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-25-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-28-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-29-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-09-30-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-01-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-02-Thu_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-03-Fri_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-05-Sun_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-06-Mon_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-07-Tue_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-08-Wed_with_grape.csv が見つかりません。
[SKIP] aim-2025-10-09-Thu_with_

特定台すべて

In [9]:
import os
import pandas as pd
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data"        # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"      # 出力先（まとめCSV）
START_DATE = "2025-10-11"   # ←開始日
END_DATE   = "2025-11-15"   # ←終了日

# 抽出条件
G_MIN = 0
RATE_MAX_DEN = 100000    # REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 10000     # 差枚 < 500

# ★★★ ここを追加・編集してください ★★★
SPECIFIC_DAI_BAN_LIST = ["593"] # 抽出したい特定の台番をリストで指定

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    return os.path.join(DATA_DIR, f"aim-{d.strftime('%Y-%m-%d')}-{weekday_str(d)}.csv")

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)          # カンマ除去
                 .str.replace(r"\s+", "", regex=True))       # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)          # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、率の分母抽出＋主要数値列を Int64 正規化して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚、台番）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator", "台番"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] < SAMAI_UPPER).fillna(False)

    # 新しく追加した台番の条件
    cond_dai_ban = df["台番"].isin(SPECIFIC_DAI_BAN_LIST).fillna(False)
    
    # 全ての条件をANDで結合
    out = df.loc[cond_g & cond_rate & cond_s & cond_dai_ban].copy()
    
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = ["台番", "差枚", "G数", "BB", "RB", "出率", "BB率", "RB率", "合成"]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(columns={c: f"{c}_next" for c in exist_cols if c != "台番"})

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)

[SKIP] aim-2025-10-12-Sun.csv が見つかりません。
[SKIP] aim-2025-10-13-Mon.csv が見つかりません。
[SKIP] aim-2025-10-14-Tue.csv が見つかりません。
[SKIP] aim-2025-10-15-Wed.csv が見つかりません。
[SKIP] aim-2025-10-16-Thu.csv が見つかりません。
[SKIP] aim-2025-10-17-Fri.csv が見つかりません。
[SKIP] aim-2025-10-19-Sun.csv が見つかりません。
[SKIP] aim-2025-10-20-Mon.csv が見つかりません。
[SKIP] aim-2025-10-21-Tue.csv が見つかりません。
[SKIP] aim-2025-10-22-Wed.csv が見つかりません。
[SKIP] aim-2025-10-23-Thu.csv が見つかりません。
[SKIP] aim-2025-10-24-Fri.csv が見つかりません。
[SKIP] aim-2025-10-26-Sun.csv が見つかりません。
[SKIP] aim-2025-10-27-Mon.csv が見つかりません。
[SKIP] aim-2025-10-28-Tue.csv が見つかりません。
[SKIP] aim-2025-10-29-Wed.csv が見つかりません。
[SKIP] aim-2025-10-30-Thu.csv が見つかりません。
[SKIP] aim-2025-10-31-Fri.csv が見つかりません。
[SKIP] aim-2025-11-02-Sun.csv が見つかりません。
[SKIP] aim-2025-11-03-Mon.csv が見つかりません。
[SKIP] aim-2025-11-04-Tue.csv が見つかりません。
[SKIP] aim-2025-11-05-Wed.csv が見つかりません。
[SKIP] aim-2025-11-06-Thu.csv が見つかりません。
[SKIP] aim-2025-11-07-Fri.csv が見つかりません。
[SKIP] aim-2025-11-09-Sun.csv が見つかりません。


In [ ]:
import pandas as pd

# ファイルの絶対パスを指定してください
file_path = r'C:\Users\akafu\analytics\aim_machi\notebook\result\filter-next-aim-20250613 20251104.csv'

try:
    # CSVファイルを読み込みます
    df = pd.read_csv(file_path)

    # 'next_date' 列を日付型に変換します
    # これにより、日付に基づいた条件での絞り込みが可能になります
    df['next_date'] = pd.to_datetime(df['next_date'])

    # 'next_date' が2025年9月のデータのみを抽出します
    start_date = '2025-09-01'
    end_date = '2025-09-30'
    september_data = df[(df['next_date'] >= start_date) & (df['next_date'] <= end_date)]

    # 抽出したデータを表示します
    print("--- 2025年9月のデータ ---")
    print(september_data)

    # 抽出したデータを新しいCSVファイルとして保存します
    # index=False は、保存時に行番号を含めないための設定です
    output_filename = 'juggler_september_2025_data.csv'
    september_data.to_csv(output_filename, index=False)
    
    print(f"\n抽出したデータを '{output_filename}' という名前で保存しました。")


except FileNotFoundError:
    print(f"エラー: ファイルが見つかりませんでした。パスを確認してください。")
    print(f"指定されたパス: {file_path}")

--- 2025年9月のデータ ---
           date  next_date   台番    差枚    G数      出率  BB  RB     合成    BB率  \
520  2025-08-31 2025-09-01  682  4131  8813  115.6%  41  44  1/104  1/215   
521  2025-08-31 2025-09-01  699  3849  8506  115.1%  42  40  1/104  1/203   
522  2025-08-31 2025-09-01  555   679  6790  103.3%  26  30  1/121  1/261   
523  2025-08-31 2025-09-01  683  2193  9335  107.8%  39  41  1/117  1/239   
524  2025-08-31 2025-09-01  675  1277  6450  106.6%  25  28  1/122  1/258   
..          ...        ...  ...   ...   ...     ...  ..  ..    ...    ...   
696  2025-09-29 2025-09-30  561  1648  8008  106.9%  31  40  1/113  1/258   
697  2025-09-29 2025-09-30  683   218  6896  101.1%  22  33  1/125  1/313   
698  2025-09-29 2025-09-30  593   -70  5428   99.6%  18  22  1/136  1/302   
699  2025-09-29 2025-09-30  674  -832  5290   94.8%  14  21  1/151  1/378   
700  2025-09-29 2025-09-30  590  2432  7428  110.9%  35  28  1/118  1/212   

       RB率  rate_denominator  差枚_next  G数_next  BB_next

In [2]:
import pandas as pd

# ファイルの絶対パスを指定してください
file_path = r'C:\Users\akafu\analytics\aim_machi\notebook\result\filter-next-aim-20250613 20251005.csv'

try:
    # CSVファイルを読み込みます
    df = pd.read_csv(file_path)

    # --- ステップ1: データを日付で絞り込む ---

    # 'next_date' 列を日付型に変換します
    df['next_date'] = pd.to_datetime(df['next_date'])

    # 'next_date' が2025年9月のデータのみを抽出します
    start_date = '2025-09-30'
    end_date = '2025-09-30'
    september_data = df[(df['next_date'] >= start_date) & (df['next_date'] <= end_date)].copy()
    
    # .copy() をつけて、以降の処理で元のデータに影響が出ないようにします

    print("--- 2025年9月のデータが抽出されました ---")
    print(september_data.head()) # 抽出したデータの先頭5行を表示


    # --- ステップ2: 絞り込んだデータで合計を計算する ---

    # '差枚_next' 列の '-' などの値を0に変換します
    september_data['差枚_next'] = pd.to_numeric(september_data['差枚_next'], errors='coerce').fillna(0)
    
    # '差枚_next' 列を整数型に変換します
    september_data['差枚_next'] = september_data['差枚_next'].astype(int)

    # '差枚_next' の合計を計算します
    total_sashimai_next_september = september_data['差枚_next'].sum()

    # 結果を出力します
    print("\n--- 計算結果 ---")
    print(f"2025年9月の『差枚_next』の合計は: {total_sashimai_next_september}")


except FileNotFoundError:
    print(f"エラー: ファイルが見つかりませんでした。パスを確認してください。")
    print(f"指定されたパス: {file_path}")

--- 2025年9月のデータが抽出されました ---
           date  next_date   台番   差枚    G数      出率  BB  RB     合成    BB率  \
392  2025-09-29 2025-09-30  683  218  6896  101.1%  22  33  1/125  1/313   
393  2025-09-29 2025-09-30  694  453  4591  103.3%  17  20  1/124  1/270   
394  2025-09-29 2025-09-30  593  -70  5428   99.6%  18  22  1/136  1/302   
395  2025-09-29 2025-09-30  674 -832  5290   94.8%  14  21  1/151  1/378   

       RB率  rate_denominator  差枚_next  G数_next  BB_next  RB_next 出率_next  \
392  1/209             209.0   1980.0   9237.0     38.0     36.0  107.1%   
393  1/230             230.0   -118.0   4381.0     17.0      5.0   99.1%   
394  1/247             247.0   1985.0   6924.0     33.0     25.0  109.6%   
395  1/252             252.0   -957.0   1302.0      1.0      4.0   75.5%   

    BB率_next RB率_next 合成_next  
392    1/243    1/257   1/125  
393    1/258    1/876   1/199  
394    1/210    1/277   1/119  
395   1/1302    1/326   1/260  

--- 計算結果 ---
2025年9月の『差枚_next』の合計は: 2890


In [11]:
import pandas as pd

# ファイルの絶対パスを指定してください
file_path = r'C:\Users\akafu\analytics\aim_machi\notebook\result\filter-next-aim-20250613 20251001.csv'

try:
    # CSVファイルを読み込みます
    df = pd.read_csv(file_path)

    # --- ステップ1: データを日付と曜日で絞り込む ---

    # 'next_date' 列を日付型に変換します
    df['next_date'] = pd.to_datetime(df['next_date'])

    # 'next_date' が2025年9月のデータのみをまず抽出します
    start_date = '2025-06-13'
    end_date = '2025-10-01'
    september_data = df[(df['next_date'] >= start_date) & (df['next_date'] <= end_date)]
    
    # さらに、その中から火曜日（dayofweek == 1）のデータだけを絞り込みます
    # 月曜日=0, 火曜日=1, 水曜日=2, 木曜日=3, 金曜日=4, 土曜日=5, 日曜日=6
    tuesday_data = september_data[september_data['next_date'].dt.dayofweek == 1].copy()


    print("--- 2025年9月の火曜日のデータが抽出されました ---")
    print(tuesday_data.head()) # 抽出したデータの先頭5行を表示


    # --- ステップ2: 絞り込んだデータで合計を計算する ---

    # '差枚_next' 列の '-' などの値を0に変換します
    tuesday_data['差枚_next'] = pd.to_numeric(tuesday_data['差枚_next'], errors='coerce').fillna(0)
    
    # '差枚_next' 列を整数型に変換します
    tuesday_data['差枚_next'] = tuesday_data['差枚_next'].astype(int)

    # '差枚_next' の合計を計算します
    total_sashimai_next_tuesday = tuesday_data['差枚_next'].sum()

    # 結果を出力します
    print("\n--- 計算結果 ---")
    print(f"2025年ある期間のx曜日の『差枚_next』の合計は: {total_sashimai_next_tuesday}")


except FileNotFoundError:
    print(f"エラー: ファイルが見つかりませんでした。パスを確認してください。")
    print(f"指定されたパス: {file_path}")

--- 2025年9月の火曜日のデータが抽出されました ---
          date  next_date   台番    差枚    G数      出率  BB  RB     合成    BB率  \
29  2025-06-16 2025-06-17  564  4539  9372  116.1%  42  50  1/102  1/223   
30  2025-06-16 2025-06-17  559  2685  5336  116.8%  26  26  1/103  1/205   
31  2025-06-16 2025-06-17  545   202  6268  101.1%  22  27  1/128  1/285   
32  2025-06-16 2025-06-17  594  3279  7846  113.9%  39  32  1/111  1/201   
33  2025-06-16 2025-06-17  561  2155  7231  109.9%  32  29  1/119  1/226   

      RB率  rate_denominator  差枚_next  G数_next  BB_next  RB_next 出率_next  \
29  1/187             187.0     97.0   1266.0      5.0      5.0  102.6%   
30  1/205             205.0   -561.0    413.0      0.0      0.0   54.7%   
31  1/232             232.0   2251.0   4367.0     24.0     15.0  117.2%   
32  1/245             245.0    876.0   4731.0     20.0     17.0  106.2%   
33  1/249             249.0  -1103.0   1589.0      2.0      4.0   76.9%   

   BB率_next RB率_next 合成_next  
29    1/253    1/253   1/127 

In [8]:
import pandas as pd

# ファイルの絶対パスを指定してください
file_path = r'C:\Users\akafu\analytics\aim_machi\notebook\result\filter-next-aim-20250613 20251104.csv'

try:
    # CSVファイルを読み込みます
    df = pd.read_csv(file_path)

    # --- ステップ1: データを日付と曜日で絞り込む ---

    # 'next_date' 列を日付型に変換します
    df['next_date'] = pd.to_datetime(df['next_date'])

    # 'next_date' が2025年9月のデータのみをまず抽出します
    start_date = '2025-06-13'
    end_date = '2025-11-04'
    september_data = df[(df['next_date'] >= start_date) & (df['next_date'] <= end_date)]
    
    # さらに、その中から火曜日（dayofweek == 1）のデータだけを絞り込みます
    # 月曜日=0, 火曜日=1, 水曜日=2, 木曜日=3, 金曜日=4, 土曜日=5, 日曜日=6
    tuesday_data = september_data[september_data['next_date'].dt.dayofweek == 6].copy()


    print("--- 2025年x月のx曜日のデータが抽出されました ---")
    print(tuesday_data.head())


    # --- ステップ2: 絞り込んだデータで各列の合計・平均を計算する ---

    # '差枚_next' 列の合計を計算
    tuesday_data['差枚_next'] = pd.to_numeric(tuesday_data['差枚_next'], errors='coerce').fillna(0)
    total_sashimai_next = tuesday_data['差枚_next'].astype(int).sum()

    # 'G数_next' 列の合計を計算
    tuesday_data['G数_next'] = pd.to_numeric(tuesday_data['G数_next'], errors='coerce').fillna(0)
    total_games_next = tuesday_data['G数_next'].astype(int).sum()

    # 'RB率_next' 列の分母の平均を計算
    # '1/257' のような文字列から '/' を基準に分割し、後ろの数字（分母）を取得
    rb_rate_denominator = tuesday_data['RB率_next'].str.split('/', expand=True)[1]
    
    # 数字に変換できないデータ（例: '-' や欠損値）は無視して、平均を計算
    avg_rb_rate_denominator = pd.to_numeric(rb_rate_denominator, errors='coerce').mean()


    # --- ステップ3: 結果を出力する ---
    print("\n--- 計算結果 ---")
    print(f"2025年x月のx曜日の『差枚_next』の合計は: {total_sashimai_next}")
    print(f"2025年x月のx曜日の『G数_next』の合計は: {total_games_next}")
    # f'{avg_rb_rate_denominator:.2f}' は、小数点以下2桁まで表示する書式設定
    print(f"2025年x月のx曜日の『RB率_next』の分母の平均は: 1/{avg_rb_rate_denominator:.2f}")


except FileNotFoundError:
    print(f"エラー: ファイルが見つかりませんでした。パスを確認してください。")
    print(f"指定されたパス: {file_path}")

--- 2025年x月のx曜日のデータが抽出されました ---
          date  next_date   台番    差枚    G数      出率  BB  RB     合成    BB率  \
10  2025-06-14 2025-06-15  696  3674  8303  114.7%  38  41  1/105  1/219   
11  2025-06-14 2025-06-15  677  2174  4661  115.5%  23  23  1/101  1/203   
12  2025-06-14 2025-06-15  587  1609  6072  108.8%  24  27  1/119  1/253   
13  2025-06-14 2025-06-15  562  1688  4786  111.8%  21  21  1/114  1/228   
14  2025-06-14 2025-06-15  548  1997  8477  107.9%  34  37  1/119  1/249   

      RB率  rate_denominator  差枚_next  G数_next  BB_next  RB_next 出率_next  \
10  1/203             203.0   3003.0   8053.0     39.0     31.0  112.4%   
11  1/203             203.0   -937.0   2410.0      7.0      4.0     87%   
12  1/225             225.0    784.0   7974.0     35.0     22.0  103.3%   
13  1/228             228.0   3557.0   7627.0     38.0     33.0  115.5%   
14  1/229             229.0   -219.0   2952.0     11.0      7.0   97.5%   

   BB率_next RB率_next 合成_next  
10    1/206    1/260   1/115 